# Setup
## Imports

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
import constants
import config

In [ ]:
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

## Config

In [ ]:
base_path = "sweeps/"

In [ ]:
offline_color = "forestgreen"
measured_color = "darkorchid"
constant_color = "dodgerblue"

## Functions

In [ ]:
def plot_bool_in_bg(x_axis, y_axis):
    ylims = plt.ylim()
    ylim_range = ylims[1] - ylims[0]
    no_val = ylims[0]+ylim_range*0.2
    yes_val = ylims[0]+ylim_range*0.8
    contacts_to_plot = np.ones_like(x_axis)*no_val
    contacts_to_plot[y_axis == 1] = yes_val
    plt.plot(x_axis, contacts_to_plot, linestyle=':', zorder=0)

In [ ]:
def draw_traj(data, sim_args, plot_manipulator=True, progs=None, num_times=5):
    w_L = sim_args['sim_params'].w_L
    h_L = sim_args['sim_params'].h_L
    r = sim_args['sim_params'].r

    # Set to be arbitrary large/small numbers
    # IMPORTANT: x and y here refer to PLOT x and ys, not sim
    min_x = 10**9
    min_y = 10**9
    max_x = -10**9
    max_y = -10**9

    # Initialize plotting vars
    fig, ax = plt.subplots(1, 1, figsize=(6,3), constrained_layout=True, sharex=True, sharey=True)
    ax.set_ylabel("$z$ position")

    # Iterate over data
    cmap = plt.get_cmap("viridis_r")
    start_idx = data["any_links_in_contact_idx"]
    start_time = data["times"][start_idx]
    end_time = data["times"][-1]
    time_range = end_time - start_time
    if progs is None:
        times_ = np.linspace(start_time, end_time, num_times)
    else:
        times_ = np.array(progs)*time_range + start_time

    # Draw rectangles over links
    for t in times_:
        c = cmap((t-start_time)/time_range)
        idx = np.argmax(data["times"] >= t)

        for x, y, ang in zip(
                data["horizontal_paper_traces"][:,idx],
                data["vertical_paper_traces"][:,idx],
                data["angle_paper_traces"][:,idx]):

            x0 = x - np.cos(ang)*w_L/2
            y0 = y - np.sin(ang)*w_L/2

            rect = plt. Rectangle((x0, y0), w_L, h_L,
                angle=ang*180/np.pi, color=c,)
            ax.add_patch(rect)

            # Compute bounds to be used
            min_x = min(x - w_L*1.1*0.5, min_x)
            min_y = min(y - w_L*1.1*0.5, min_y)
            max_x = max(x + w_L*1.1*0.5, max_x)
            max_y = max(y + w_L*1.1*0.5, max_y)

    # Add pedestal
    ## Get first pedestal location
    rect = plt.Rectangle(
        (data["horizontal_paper_traces"][0,0]+w_L/2,
         data["vertical_paper_traces"][0,0]),
        w_L,
        10,
        angle=180,
        color=(0.2, 0.2, 0.2, 1.0)
    )
    ax.add_patch(rect)

    # Add manipulator
    if plot_manipulator:
        for t in times_:
            c = cmap((t-start_time)/time_range)
            idx = np.argmax(data_["times"] >= t)

            x = data["horizontal_manipulator_trace"][idx]
            y = data["vertical_manipulator_trace"][idx]

            min_x = min(x - r*1.1, min_x)
            min_y = min(y - r*1.1, min_y)
            max_x = max(x + r*1.1, max_x)
            max_y = max(y + r*1.1, max_y)

            circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
            ax.add_patch(circ)

    ax.set_xlabel("$y$ position")
    ax.set_aspect("equal")

    # Set lims (can do at the end because sharex and sharey are true)
    min_plot_x = min_x - (max_x-min_x)*0.1
    max_plot_x = max_x + (max_x-min_x)*0.1
    min_x_tick = np.floor(min_plot_x*10)/10
    max_x_tick = np.ceil(max_plot_x*10)/10
    xticks = np.arange(min_x_tick, max_x_tick+1e-5, 0.1)
    ax.set_xticks(xticks)

    min_plot_y = min_y - (max_y-min_y)*0.1
    max_plot_y = max_y + (max_y-min_y)*0.1
    min_y_tick = np.floor(min_plot_y*10)/10
    max_y_tick = np.ceil(max_plot_y*10)/10
    yticks = np.arange(min_y_tick, max_y_tick+1e-5, 0.1)
    ax.set_yticks(yticks)
    ax.set_ylim(min_plot_y, max_plot_y)
    plt.xlim(min_plot_x, max_plot_x)
    cb = fig.colorbar(cm.ScalarMappable(cmap="viridis_r"))
    cb.set_label("Progress")
    ticks = np.linspace(0, 100, 6)
    cb.set_ticks(ticks/100)
    cb.set_ticklabels([str(int(tick)) + "\%" for tick in ticks])
    
    return fig, ax

In [ ]:
def draw_trajs(datas, sim_args, plot_manipulator=True, progs=None, num_times=5,
              fig_height=2.1, cb_shrink=0.8):
    w_L = sim_args['sim_params'].w_L
    h_L = sim_args['sim_params'].h_L
    r = sim_args['sim_params'].r

    # Set to be arbitrary large/small numbers
    # IMPORTANT: x and y here refer to PLOT x and ys, not sim
    min_x = 10**9
    min_y = 10**9
    max_x = -10**9
    max_y = -10**9

    # Initialize plotting vars
    fig, axs = plt.subplots(1, len(datas), figsize=(6,fig_height), constrained_layout=True, sharex=True, sharey=True)
    axs[0].set_ylabel("$z$ position")
    cmap = plt.get_cmap("viridis_r")
    for i, data_, in enumerate(datas):
        start_idx = data_["any_links_in_contact_idx"]
        start_time = data_["times"][start_idx]
        end_time = data_["times"][-1]
        time_range = end_time - start_time
        
        if progs is None:
            times_ = np.linspace(start_time, end_time, num_times)
        else:
            if type(progs[0]) is list:
                progs_ = np.array(progs[i])
            else:
                progs_ = np.array(progs)
            times_ = progs_*time_range + start_time
        ax = axs[i]

        # Draw rectangles over links
        for t in times_:
            c = cmap((t-start_time)/time_range)
            idx = np.argmax(data_["times"] >= t)

            for x, y, ang in zip(
                    data_["horizontal_paper_traces"][:,idx],
                    data_["vertical_paper_traces"][:,idx],
                    data_["angle_paper_traces"][:,idx]):

                x0 = x - np.cos(ang)*w_L/2
                y0 = y - np.sin(ang)*w_L/2

                rect = plt. Rectangle((x0, y0), w_L, h_L,
                    angle=ang*180/np.pi, color=c,)
                ax.add_patch(rect)

                # Compute bounds to be used
                min_x = min(x - w_L*1.1*0.5, min_x)
                min_y = min(y - w_L*1.1*0.5, min_y)
                max_x = max(x + w_L*1.1*0.5, max_x)
                max_y = max(y + w_L*1.1*0.5, max_y)

        # Add pedestal
        ## Get first pedestal location
        rect = plt.Rectangle(
            (data_["horizontal_paper_traces"][0,0]+w_L/2,
             data_["vertical_paper_traces"][0,0]),
            w_L,
            10,
            angle=180,
            color=(0.2, 0.2, 0.2, 1.0)
        )
        ax.add_patch(rect)

        # Add manipulator
        if plot_manipulator:
            for t in times_:
                c = cmap((t-start_time)/time_range)
                idx = np.argmax(data_["times"] >= t)

                x = data_["horizontal_manipulator_trace"][idx]
                y = data_["vertical_manipulator_trace"][idx]

                min_x = min(x - r*1.1, min_x)
                min_y = min(y - r*1.1, min_y)
                max_x = max(x + r*1.1, max_x)
                max_y = max(y + r*1.1, max_y)

                circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
                ax.add_patch(circ)

        ax.set_xlabel("$y$ position")
        ax.set_aspect("equal")

    # Set lims (can do at the end because sharex and sharey are true)
    min_plot_x = min_x - (max_x-min_x)*0.1
    max_plot_x = max_x + (max_x-min_x)*0.1
    min_x_tick = np.floor(min_plot_x*10)/10
    max_x_tick = np.ceil(max_plot_x*10)/10
    xticks = np.arange(min_x_tick, max_x_tick+1e-5, 0.1)
    ax.set_xticks(xticks)

    min_plot_y = min_y - (max_y-min_y)*0.1
    max_plot_y = max_y + (max_y-min_y)*0.1
    min_y_tick = np.floor(min_plot_y*10)/10
    max_y_tick = np.ceil(max_plot_y*10)/10
    yticks = np.arange(min_y_tick, max_y_tick+1e-5, 0.1)
    ax.set_yticks(yticks)
    ax.set_ylim(min_plot_y, max_plot_y)
    plt.xlim(min_plot_x, max_plot_x)
    cb = fig.colorbar(cm.ScalarMappable(cmap="viridis_r"), shrink=cb_shrink)
    cb.set_label("Progress")
    ticks = np.linspace(0, 100, 6)
    cb.set_ticks(ticks/100)
    cb.set_ticklabels([str(int(tick)) + "\%" for tick in ticks])

    return fig, axs, cb

# Intro sweeps

In [ ]:
data = np.load(base_path + "traces.npz", allow_pickle=True)

In [ ]:
plot_manipulator=True
data = np.load(base_path + "traces.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(data["out_data"], sim_args, plot_manipulator=plot_manipulator,
               fig_height=3)
titles = ["Offline setpoint\ntrajectory",
          r"Link feedback with" + "\n" +
          "constant $F_{Nff}$",
          "Link feedback with" + "\n" +
          "$F_{Nff}$ based on" + "\n" +
          "force measurements"]
for title, ax in zip(titles, axs):
    ax.set_title(title)
plt.suptitle("System behavior with various strategies")
# fig.savefig("traj_drawings.pgf", backend="pgf")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)
colors = [offline_color, constant_color, measured_color]
titles = ["Offline setpoint trajectory",
          r"Link feedback with constant $F_{Nff}$",
          "Link feedback with $F_{Nff}$ based on force measurements"]
for i, (data_, color, title) in enumerate(zip(data['out_data'], colors, titles)):
    start_idx = data_["any_links_in_contact_idx"]
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=color, label=title)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=color)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=color)
plt.legend(frameon=False)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.gca().set_aspect("equal")

fig.savefig("traj_plots.pgf", backend="pgf")
plt.show()

# Task parameters

# Control parameters

## Feedforward $F_N$

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__ff_FN.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__ff_FN.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [measured_color, constant_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    if i == 1:
        x_axis = data["sweep_vars"]
        y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    else:
        x_axis = data["x_axis"]
        y_axis = data["y_axis"]
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.plot(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Additional $F_n$ (N)")
plt.ylabel("Angle about joint (radians)")
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__ff_FN.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(data['out_data'], data['sweep_vars'].flatten())):
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Additional $F_n$ (N)")
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__ff_FN.npz", allow_pickle=True)
idxs = [2, -6, -1]
Fns = data["sweep_vars"][idxs]
datas = data["out_data"][idxs]
sim_args = data["sim_args"][0]


fig, axs, cb = draw_trajs(datas, sim_args, progs=[
    [0, 0.2, 1],
    [0,0.04, 0.4,1],
    [0, 0.2, 0.4, 1]
])

for Fn, ax in zip(Fns, axs):
    ax.set_title("$F_N$ = " + str(Fn[0]))
plt.suptitle("System behavior with constant feedforward $F_N$")
plt.show()

## Link mass

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__m_L.npz", allow_pickle=True),
    np.load(base_path + "offline__m_L.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant", "Offline trajectory"]
colors = [measured_color, constant_color, offline_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.plot(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Link mass ($m_L$)")
plt.ylabel("Angle about joint (radians)")
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__m_L.npz", allow_pickle=True)
idxs = [0, 3, -1]
m_Ls = data["sweep_vars"][idxs]
datas = data["out_data"][idxs]
sim_args = data["sim_args"][0]


fig, axs, cb = draw_trajs(datas, sim_args)

for m_L, ax in zip(m_Ls, axs):
    ax.set_title("$m_L$ = " + "{:.2f} kg".format(m_L[0]))

In [ ]:
data = np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True)
idxs = [0, 6, -1]
m_Ls = data["sweep_vars"][idxs]
datas = data["out_data"][idxs]
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(datas, sim_args)

for m_L, ax in zip(m_Ls, axs):
    ax.set_title("$m_L$ = " + "{:.2f} kg".format(m_L[0]))

In [ ]:
data = np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True)
idxs = list(range(8)) + [9, -1]
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'][idxs])))
for i, (data_, Fn_noise) in enumerate(zip(data['out_data'][idxs], data['sweep_vars'].flatten()[idxs])):
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")


cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Link mass (kg)")
plt.show()

## Stiffness

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__k_J.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__k_J.npz", allow_pickle=True),
    np.load(base_path + "offline__k_J.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant", "Offline trajectory"]
colors = [measured_color, constant_color, offline_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"][:,0]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.plot(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Link stiffness (Nm/rad)")
plt.ylabel("Angle about joint (radians)")
plt.show()

- [ ] TODO: that one failure is just due to the lockin firing prematurely. Fix that

In [ ]:
data = np.load(base_path + "measured_Fn__k_J.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, k_J) in enumerate(zip(data['out_data'][3:], data['sweep_vars'][3:,0])):
    c = color_map.to_rgba(k_J)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Additional $F_n$ (N)")
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__k_J.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(data['out_data'], data['sweep_vars'][:,0])):
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Additional $F_n$ (N)")
plt.show()

# Coefficient of friction

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__mu.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__mu.npz", allow_pickle=True),
    np.load(base_path + "offline__mu.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant", "Offline trajectory"]
colors = [measured_color, constant_color, offline_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.plot(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Coefficient of friction")
plt.ylabel("Angle about joint (radians)")
plt.show()

# Damping

In [ ]:
# # data1 = np.load(base_path + "constant_Fn__b_J.npz")
# data2 = np.load(base_path + "measured_Fn__b_J.npz", allow_pickle=True)
# data3 = np.load(base_path + "offline__b_J.npz", allow_pickle=True)

# # plt.plot(data1["x_axis"][:,0], data1["y_axis"], '--o', label="Constant")
# # plot_bool_in_bg(data1["x_axis"][:,0], data1["successes"])

# plt.plot(data2["x_axis"][:,0], data2["y_axis"], '--o', label="Measured")
# plot_bool_in_bg(data2["x_axis"][:,0], data2["successes"])

# # plt.plot(data3["x_axis"][:,0], data3["y_axis"], '--o', label="Offline")
# # plot_bool_in_bg(data3["x_axis"][:,0], data3["successes"])

# plt.legend(frameon=False)
# plt.show()

# $F_n$ Noise

In [ ]:
data = np.load(base_path + "measured_Fn__Fn_noise.npz", allow_pickle=True)

In [ ]:
x_axis = data["sweep_vars"][:,0]
y_axis = [val["max_overall_theta"] for val in data["out_data"]]
plt.plot(x_axis, y_axis, '--o')
plot_bool_in_bg(data["sweep_vars"][:,0], data["successes"])

plt.ylim(0, np.pi)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(data['out_data'], data['sweep_vars'].flatten())):
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("$F_N$ noise")
plt.show()

In [ ]:
plt.figure()
plt.plot(data["out_data"][0]["times"], data["out_data"][0]["max_F_ONs"])
plt.show()

# Link noise

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__link_noise.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__link_noise.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [measured_color, constant_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"][:,0]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.semilogx(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Coefficient of friction")
plt.ylabel("Angle about joint (radians)")
plt.show()